In [1]:
import time
from design import *
import importlib
import shutil
from utils import *
from openai import OpenAI
from prompts import *
import json
import numpy as np
from gymnasium.envs.robodesign.GPTAnt import GPTAntEnv

In [ ]:
import prompts
class DGA:
    def __init__(self):
        self.client = OpenAI(api_key=api_key)
        self.model = "gpt-4o-mini"

    def extract_code(self, text):
        match = re.search(r'```python\n(.*?)\n```', text, re.DOTALL)
        return match.group(1).strip() if match else None

    def indent_code(self, code):
        return "\n".join(line if line.strip() else line for line in code.split("\n"))

    def generate_rewardfunc(self, rewardfunc_nums, folder_name):

        messages = [
            {"role": "system", "content": "You are a reinforcement learning reward function designer"},
            {"role": "user", "content": rewardfunc_prompts + zeroshot_rewardfunc_format}
        ]

        responses = self.client.chat.completions.create(
            model=self.model, messages=messages, n=rewardfunc_nums
        )
        files = []
        for i, choice in enumerate(responses.choices):
            reward_code = self.extract_code(choice.message.content)
            if reward_code:
                full_code = self.indent_code(reward_code) + "\n"
                file_name =  f"GPTAnt_{i}.py"
                file_path = os.path.join(folder_name, "env", file_name)
                with open(file_path, "w") as fp:
                    fp.write(full_code)

                with open(file_path, "w") as fp:
                    fp.write(full_code)
                files.append(file_path)
                print(f"Saved: {file_path}")
        return files
    
    def generate_rewardfunc_div(self, rewardfunc_nums, folder_name):

        # env_path = os.path.join(os.path.dirname(__file__), "env", "ant_v5.py")
        # with open(env_path, "r") as f:
        #     env_content = f.read().rstrip()

        messages = [
            {"role": "system", "content": "You are a reinforcement learning reward function designer"},
            {"role": "user", "content": rewardfunc_prompts + zeroshot_rewardfunc_format}
        ]

        # 生成初始 Reward Function
        response = self.client.chat.completions.create(
            model=self.model, messages=messages, n=1, timeout=10
        )

        rewardfunc_files = []

        initial_code = self.extract_code(response.choices[0].message.content)
        if initial_code:
            reward_code = "import numpy as np\n" + self.indent_code(initial_code) + "\n"

            file_path = os.path.join(folder_name, "env", "GPTrewardfunc_0.py")
            with open(file_path, "w") as fp:
                fp.write(reward_code)
            rewardfunc_files.append(file_path)
            print(f"initial Saved: {file_path}")

        # 生成不同的多样化 Reward Functions
        for i in range(1, rewardfunc_nums):
            diverse_messages = messages + [
                {"role": "user", "content": rewardfunc_div_prompts + zeroshot_rewardfunc_format}
            ]

            response = self.client.chat.completions.create(
                model=self.model, messages=diverse_messages, n=1
            )

            diverse_code = self.extract_code(response.choices[0].message.content)
            if diverse_code:
                reward_code =  "import numpy as np\n" + self.indent_code(diverse_code) + "\n"
                file_path = os.path.join(folder_name, "env", f"GPTrewardfunc_{i}.py")
                with open(file_path, "w") as fp:
                    fp.write(reward_code)
                rewardfunc_files.append(file_path)
                print(f"Saved: {file_path}")

        return rewardfunc_files

    def generate_morphology(self, morphology_nums, folder_name):
        messages = [
            {"role": "system", "content": "You are a helpful mujoco robot designer"},
            {"role": "user", "content": morphology_prompts + morphology_format}
        ]
        
        responses = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            response_format={'type': 'json_object'},
            n=morphology_nums
        )

        # 解析所有 response 里的参数
        for i, choice in enumerate(responses.choices):
            print(f"Response {i}:")
            print(json.dumps(choice.message.content, indent=4))

        parameter_list = [json.loads(choice.message.content).get('parameters', []) for choice in responses.choices]
        material_list = [compute_ant_volume(parameter) for parameter in parameter_list]

        xml_files = []
        for i, parameter in enumerate(parameter_list):
            if not isinstance(parameter, list):
                print(f"Skipping invalid parameter {i}: {parameter}")
                continue

            xml_file = ant_design(parameter)  
            filename = f"GPTAnt_{i}.xml"
            file_path = os.path.join(folder_name, "assets", filename)
            xml_files.append(file_path)
            with open(file_path, "w") as fp:
                fp.write(xml_file)
            print(f"Successfully saved {filename}")
            
        return xml_files, material_list, parameter_list
    
    def generate_morphology_div(self, morphology_nums, folder_name):

        material_list = []
        xml_files = []
        parameter_list = []
        
        # 生成初始 morphology
        messages = [
            {"role": "system", "content": "You are a helpful mujoco robot designer"},
            {"role": "user", "content": morphology_prompts + morphology_format}
        ]
        
        response = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            response_format={'type': 'json_object'},
            n=1
        )
        

        initial_parameter = json.loads(response.choices[0].message.content)
        parameter_list.append(initial_parameter['parameters'])
        material_list.append(compute_ant_volume(initial_parameter['parameters']))
        messages.append({"role": "assistant", "content": json.dumps(initial_parameter)})

        logging.info(f"generate initial_parameter{initial_parameter['parameters']}" )

        xml_file = ant_design(initial_parameter['parameters'])  

        filename = f"GPTAnt_0.xml"
        file_path = os.path.join(folder_name, "assets", filename)
        with open(file_path, "w") as fp:
            fp.write(xml_file)

        xml_files.append(file_path)

        # 生成不同的多样化设计
        for i in range(1, morphology_nums):
            diverse_messages = messages + [
                {"role": "user", "content": morphology_div_prompts + morphology_format}
            ]
            
            response = self.client.chat.completions.create(
                model=self.model,
                messages=diverse_messages,
                response_format={'type': 'json_object'},
                n=1
            )

            diverse_parameter = json.loads(response.choices[0].message.content)
            material_list.append(compute_ant_volume(diverse_parameter['parameters']))
            parameter_list.append(diverse_parameter['parameters'])
            messages.append({"role": "assistant", "content": json.dumps(diverse_parameter)})
            logging.info(f"generate diverse_parameter{ diverse_parameter['parameters']}")
            xml_file = ant_design(diverse_parameter['parameters'])  
            filename = f"GPTAnt_{i}.xml"
            file_path = os.path.join(folder_name, "assets", filename)
            with open(file_path, "w") as fp:
                fp.write(xml_file)
            xml_files.append(file_path)

        return xml_files, material_list, parameter_list


    def improve_rewardfunc(self, best_rewardfunc, rewardfunc_list, fitness_list, folder_name, step, rewardfunc_index, morphology_index):
        reward_improve_prompts = prompts.reward_improve_prompts
        for reward_content, fitness in zip(rewardfunc_list, fitness_list):
            reward_improve_prompts = reward_improve_prompts + f"reward function:{reward_content} \n" + f"fintess:{fitness}"
        reward_improve_prompts = reward_improve_prompts + f"best reward function:{best_rewardfunc} \n" + f"best fintess:{max(fitness_list)}" 

        messages = [
            {"role": "system", "content": "You are a reinforcement learning reward function designer"},
            {"role": "user", "content": rewardfunc_prompts + rewardfunc_format}
        ]

        response = self.client.chat.completions.create(
            model=self.model, messages=messages
        )

        print(response)
        reward_code = self.extract_code(response.choices[0].message.content)

        if reward_code:
            full_code = "import numpy as np \n" + self.indent_code(reward_code) + "\n"
            file_name =  f"GPTAnt_refine_{step}_{rewardfunc_index}_{morphology_index}.py"
            file_path = os.path.join(folder_name, "env", file_name)
            with open(file_path, "w") as fp:
                fp.write(full_code)

        return file_path
    
    

    def improve_morphology(self, best_parameter, parameter_list, fitness_list, folder_name, step, rewardfunc_index, morphology_index):
        morphology_improve_prompts = prompts.morphology_improve_prompts
        for parameter_content, fitness in zip(parameter_list, fitness_list):
            morphology_improve_prompts = morphology_improve_prompts + f"parameter:{parameter_content} \n" + f"fintess:{fitness}"
        morphology_improve_prompts = morphology_improve_prompts + f"best parameter:{best_parameter} \n" + f"best fintess:{max(fitness_list)}" 

        messages = [
            {"role": "system", "content": "You are a helpful mujoco robot designer"},
            {"role": "user", "content": morphology_improve_prompts + morphology_format}
        ]
        
        responses = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            response_format={'type': 'json_object'},
        )
        print(responses)
        parameter = json.loads(responses.choices[0].message.content).get('parameters', []) 
        print(parameter)
        xml_file = ant_design(parameter)  
        filename = f"GPTAnt_refine_{step}_{rewardfunc_index}_{morphology_index}.xml"
        file_path = os.path.join(folder_name, "assets", filename)

        with open(file_path, "w") as fp:
            fp.write(xml_file)

        print(f"Successfully saved {filename}")
        return file_path, parameter


# Configuration

In [3]:

folder_name = "results/noDiv_m25_r5"
log_file = os.path.join(folder_name, "parameters.log")
logging.basicConfig(filename=log_file, level=logging.INFO, format="%(asctime)s - %(message)s")

# folder_name = setup_logging(div_flag=True)

best_fitness = float('-inf')  
best_morphology = None  
best_rewardfunc = None  
best_reward = None
best_material = None
best_efficiency = None

morphology_nums = 25
rewardfunc_nums = 5

fitness_matrix = np.array([[None for _ in range(morphology_nums)] for _ in range(rewardfunc_nums)])
efficiency_matrix = np.array([[None for _ in range(morphology_nums)] for _ in range(rewardfunc_nums)])
fitness_list = []
designer = DGA()



# return file list of morphology and reward function: [GPTAnt_{i}.xml] and [GPTAnt_{j}.py]



In [4]:
logging.info(f"start!")

# print configuration info

In [7]:
designer = DGA()
morphology_list, material_list, parameter_list = designer.generate_morphology_div(morphology_nums, folder_name)

params: [0.2, 0.3, 0.06, 0.2, 0.15, 0.1, 0.05, 0.02, 0.02, 0.015]
params: [0.25, 0.35, 0.08, 0.25, 0.2, 0.15, 0.1, 0.03, 0.025, 0.02]
params: [0.15, 0.2, 0.05, 0.15, 0.1, 0.08, 0.04, 0.015, 0.015, 0.01]
params: [0.3, 0.45, 0.1, 0.4, 0.3, 0.25, 0.15, 0.05, 0.04, 0.03]
params: [0.18, 0.25, 0.04, 0.12, 0.09, 0.06, 0.03, 0.02, 0.018, 0.015]
params: [0.35, 0.5, 0.1, 0.25, 0.05, 0.15, 0.02, 0.04, 0.03, 0.025]
params: [0.45, 0.6, 0.15, 0.35, 0.2, 0.25, 0.1, 0.07, 0.06, 0.05]
params: [0.2, 0.1, 0.3, 0.2, 0.4, 0.15, 0.35, 0.015, 0.025, 0.02]
params: [0.22, 0.18, 0.07, 0.1, 0.05, 0.12, 0.06, 0.025, 0.02, 0.015]
params: [0.1, 0.15, 0.02, 0.3, 0.1, 0.05, 0.02, 0.01, 0.015, 0.01]
params: [0.4, 0.25, 0.06, 0.2, 0.05, 0.3, 0.15, 0.03, 0.025, 0.02]
params: [0.25, 0.1, 0.2, 0.15, 0.3, 0.1, 0.2, 0.02, 0.03, 0.02]
params: [0.3, 0.15, 0.1, 0.05, 0.2, 0.08, 0.04, 0.02, 0.015, 0.01]
params: [0.5, 0.2, 0.12, 0.25, 0.08, 0.3, 0.1, 0.04, 0.03, 0.025]
params: [0.15, 0.25, 0.08, 0.12, 0.07, 0.15, 0.09, 0.015, 0.

In [8]:
designer = DGA()
rewardfunc_list = designer.generate_rewardfunc_div(rewardfunc_nums, folder_name)

initial Saved: results/div2025-03-21_15-03-44/env/GPTrewardfunc_0.py
Saved: results/div2025-03-21_15-03-44/env/GPTrewardfunc_1.py
Saved: results/div2025-03-21_15-03-44/env/GPTrewardfunc_2.py
Saved: results/div2025-03-21_15-03-44/env/GPTrewardfunc_3.py
Saved: results/div2025-03-21_15-03-44/env/GPTrewardfunc_4.py
Saved: results/div2025-03-21_15-03-44/env/GPTrewardfunc_5.py
Saved: results/div2025-03-21_15-03-44/env/GPTrewardfunc_6.py
Saved: results/div2025-03-21_15-03-44/env/GPTrewardfunc_7.py
Saved: results/div2025-03-21_15-03-44/env/GPTrewardfunc_8.py
Saved: results/div2025-03-21_15-03-44/env/GPTrewardfunc_9.py


In [9]:
efficiency_matrix.shape

(10, 50)

# enter coarse optimization stage

In [11]:
morphology_list = [f'results/noDiv_m25_r5/assets/GPTAnt_{i}.xml' for i in range(0,25) ]
rewardfunc_list = [f'results/noDiv_m25_r5/env/GPTrewardfunc_{i}.py' for i in range(0,5)]

parameter_list = [[0.3, 0.25, 0.1, 0.4, 0.1, 0.3, 0.1, 0.045, 0.045, 0.045],
 [0.25, 0.1, 0.05, 0.2, 0.1, 0.15, 0.075, 0.02, 0.02, 0.015],
 [0.15, 0.2, 0.05, 0.15, 0.1, 0.2, 0.1, 0.025, 0.025, 0.025],
 [0.3, 0.3, 0.15, 0.25, 0.15, 0.25, 0.15, 0.05, 0.05, 0.05],
 [0.4, 0.3, 0.2, 0.2, 0.3, 0.1, 0.2, 0.05, 0.03, 0.02],
 [0.1, 0.2, 0.1, 0.3, 0.1, 0.2, 0.1, 0.05, 0.05, 0.05],
 [0.2, 0.1, 0.075, 0.15, 0.1, 0.1, 0.075, 0.025, 0.025, 0.02],
 [0.3, 0.15, 0.15, 0.2, 0.15, 0.15, 0.15, 0.04, 0.04, 0.04],
 [0.5, 0.3, 0.3, 0.5, 0.3, 0.2, 0.2, 0.02, 0.02, 0.02],
 [0.08, 0.1, 0.04, 0.15, 0.05, 0.2, 0.05, 0.025, 0.025, 0.02],
 [0.15, 0.1, 0.07, 0.2, 0.15, 0.15, 0.1, 0.02, 0.02, 0.02],
 [0.5, 0.3, 0.15, 0.4, 0.2, 0.25, 0.15, 0.05, 0.04, 0.03],
 [0.3, 0.4, 0.2, 0.3, 0.2, 0.25, 0.15, 0.08, 0.08, 0.05],
 [0.25, 0.1, 0.06, 0.2, 0.15, 0.2, 0.15, 0.035, 0.035, 0.035],
 [0.3, 0.15, 0.05, 0.1, 0.02, 0.05, 0.02, 0.025, 0.02, 0.02],
 [0.3, 0.2, 0.15, 0.3, 0.2, 0.3, 0.15, 0.05, 0.03, 0.03],
 [0.3, 0.2, 0.1, 0.15, 0.15, 0.1, 0.1, 0.025, 0.025, 0.025],
 [0.25, 0.6, 0.15, 0.4, 0.6, 0.3, 0.15, 0.08, 0.08, 0.08],
 [0.1, 0.2, 0.05, 0.15, 0.05, 0.2, 0.05, 0.02, 0.02, 0.02],
 [0.3, 0.2, 0.1, 0.3, 0.2, 0.2, 0.1, 0.04, 0.04, 0.03],
 [0.2, 0.4, 0.3, 0.6, 0.3, 0.4, 0.3, 0.05, 0.05, 0.05],
 [0.25, 0.3, 0.2, 0.5, 0.4, 0.3, 0.2, 0.07, 0.07, 0.06],
 [0.25, 0.15, 0.05, 0.15, 0.05, 0.1, 0.05, 0.02, 0.02, 0.02],
 [0.3, 0.1, 0.15, 0.3, 0.2, 0.3, 0.2, 0.05, 0.05, 0.05],
 [0.15, 0.1, 0.06, 0.2, 0.12, 0.15, 0.1, 0.02, 0.014, 0.016]]


material_list = [compute_ant_volume(parameter) for parameter in parameter_list]

params: [0.3, 0.25, 0.1, 0.4, 0.1, 0.3, 0.1, 0.045, 0.045, 0.045]
params: [0.25, 0.1, 0.05, 0.2, 0.1, 0.15, 0.075, 0.02, 0.02, 0.015]
params: [0.15, 0.2, 0.05, 0.15, 0.1, 0.2, 0.1, 0.025, 0.025, 0.025]
params: [0.3, 0.3, 0.15, 0.25, 0.15, 0.25, 0.15, 0.05, 0.05, 0.05]
params: [0.4, 0.3, 0.2, 0.2, 0.3, 0.1, 0.2, 0.05, 0.03, 0.02]
params: [0.1, 0.2, 0.1, 0.3, 0.1, 0.2, 0.1, 0.05, 0.05, 0.05]
params: [0.2, 0.1, 0.075, 0.15, 0.1, 0.1, 0.075, 0.025, 0.025, 0.02]
params: [0.3, 0.15, 0.15, 0.2, 0.15, 0.15, 0.15, 0.04, 0.04, 0.04]
params: [0.5, 0.3, 0.3, 0.5, 0.3, 0.2, 0.2, 0.02, 0.02, 0.02]
params: [0.08, 0.1, 0.04, 0.15, 0.05, 0.2, 0.05, 0.025, 0.025, 0.02]
params: [0.15, 0.1, 0.07, 0.2, 0.15, 0.15, 0.1, 0.02, 0.02, 0.02]
params: [0.5, 0.3, 0.15, 0.4, 0.2, 0.25, 0.15, 0.05, 0.04, 0.03]
params: [0.3, 0.4, 0.2, 0.3, 0.2, 0.25, 0.15, 0.08, 0.08, 0.05]
params: [0.25, 0.1, 0.06, 0.2, 0.15, 0.2, 0.15, 0.035, 0.035, 0.035]
params: [0.3, 0.15, 0.05, 0.1, 0.02, 0.05, 0.02, 0.025, 0.02, 0.02]
params: 

In [6]:
logging.info(f'folder_name:{folder_name}')
logging.info(f'morphology_nums:{morphology_nums}')
logging.info(f'rewardfunc_nums:{rewardfunc_nums}')
logging.info(f'parameter_list:{parameter_list}')
logging.info(f'morphology_list:{morphology_list}')
logging.info(f'material_list:{material_list}')
logging.info(f'_________________________________enter coarse optimization stage_________________________________')

In [ ]:
for i, rewardfunc in enumerate(rewardfunc_list):
    for j, morphology in enumerate(morphology_list):
        # if i not in [0] or j not in [12]:
        #     continue
        # if i not in [5]:
        #     continue
        print(i, rewardfunc)
        print(j, morphology)
        shutil.copy(morphology, "GPTAnt.xml")
        shutil.copy(rewardfunc, "GPTrewardfunc.py")         

        import GPTrewardfunc
        importlib.reload(GPTrewardfunc)  # 重新加载模块
        from GPTrewardfunc import _get_rew
        GPTAntEnv._get_rew = _get_rew

        env_name = "GPTAntEnv"
        model_path = Train(j,  i, folder_name, total_timesteps=5e5)
        # model_path = f"results/div2025-03-17_15-13-46/SAC_morphology{j}_rewardfunc{i}_500000.0steps"
        fitness, reward = Eva(model_path)
        material = material_list[j]
        efficiency = fitness/material
        fitness_matrix[i][j] = fitness
        efficiency_matrix[i][j] = efficiency
        
        logging.info("___________________finish coarse optimization_____________________")
        logging.info(f"morphology: {j}, rewardfunc: {i}, material cost: {material} reward: {reward} fitness: {fitness} efficiency: {efficiency}")

        if fitness > best_fitness:
            best_fitness = fitness
            best_morphology = morphology
            best_efficiency = efficiency
            best_rewardfunc = rewardfunc
            best_material = material

0 results/noDiv_m25_r5/env/GPTrewardfunc_0.py
0 results/noDiv_m25_r5/assets/GPTAnt_0.xml
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0 results/noDiv_m25_r5/env/GPTrewardfunc_0.py
23 results/noDiv_m25_r5/assets/GPTAnt_23.xml
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0 results/noDiv_m25_r5/env/GPTrewardfunc_0.py
24 results/noDiv_m25_r5/assets/GPTAnt_24.xml
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52


In [13]:
fitness_matrix

array([[8.770676292790775, 0.4154285773900174, 0.6669313489301868,
        3.901153202958363, 0.7989037016497241, 5.258522669301234,
        0.3689786789309141, 0.49746411622571435, 0.6603007374610418,
        1.2599398827542083, 1.2487093223031756, 1.5352007262536747,
        4.238860877662644, 0.7311960637184372, 0.32300784765761614,
        5.267268249581435, 0.380499785209238, 2.817979545958534,
        2.099531828682503, 2.4811259014954157, 10.141889571996268,
        8.74535362544958, 0.33357735222421314, 5.174848340361833,
        0.7562117701403028],
       [6.7719743095361205, 0.39103667688223853, 0.4453263817322554,
        1.0094524060206356, 0.45782031736432643, 0.8502609132128915,
        0.3473546785569741, 0.44360821860449123, 0.659905064626487,
        0.42787486526315943, 0.4276842865470231, 0.6424778796124077,
        2.3742768434424413, 0.5258727347724826, 0.32466956159387295,
        5.252853170558212, 0.3651001603382401, 3.102345880235173,
        0.427874865263159

# print coarse optimization info

In [15]:
logging.info(f'_________________________________end coarse optimization stage_________________________________')
logging.info(f"Stage1: Final best morphology: {best_morphology}, Fitness: {best_fitness}, best_efficiency: {best_efficiency}, best reward function: {best_rewardfunc}, Material cost: {best_material}, Reward: {best_reward}")
logging.info(f'folder_name:{folder_name}')
logging.info(f'parameter_list:{parameter_list}')
logging.info(f'fitness_matrix:{fitness_matrix}')
logging.info(f'efficiency_matrix:{efficiency_matrix}')
logging.info(f'_________________________________enter fine optimization stage_________________________________')

In [5]:
efficiency_matrix = np.array([[61.303988615119536, 6.115124311666828, 33.83067552402663,
        26.31713922961262, 2.7808041916954376, 152.61473119498655,
        9.920401861242052, 3.830441840293891, 1.2447086934054739,
        212.60239053178248, 72.11440920910812, 2.791049744249617,
        20.529868691302045, 9.48409393371544, 2.792921029137332,
        39.95752835320133, 3.2132534724025703, 12.474667503692158,
        281.5072593591731, 19.09019617200925, 109.89882336510979,
        55.17476931449055, 4.9052975322646875, 35.03657748950116,
        46.89271454362332],
       [47.33375421857993, 5.756074617156185, 22.589569896270994,
        6.809755508901006, 1.5935696066260605, 24.67657722065737,
        9.339016578551236, 3.4157548772219326, 1.2439628250624997,
        72.19965051392703, 24.699262751935528, 1.1680477288166309,
        11.499219540215295, 6.82091529377074, 2.807289211924958,
        39.84817547776541, 3.0832063606460682, 13.733503989724285,
        57.36987599968518, 4.5546204018894025, 66.09925624064623,
        53.57561281964338, 4.904498023785656, 22.351386136944118,
        25.18833781360363],
       [58.86812383019806, 5.957039842276172, 22.72103965215382,
        15.02726243126152, 1.9710861073885395, 33.777352926488895,
        9.465702556899867, 3.3870906493884814, 1.1624722169568529,
        78.61502994084613, 47.216131972601744, 2.0117054837916704,
        13.384014729808158, 9.126823894060667, 2.783352649298262,
        48.86923807804809, 3.21541908650412, 13.187301747283003,
        62.24938381312568, 10.140527990722356, 109.06846424083525,
        50.62838784064628, 4.904498023785656, 27.03733437091659,
        32.38598876940104],
       [56.27424834364141, 5.8445335663076765, 23.15057050401981,
        10.77006895457607, 1.96321316833573, 26.09318601724348,
        9.376630640595804, 3.379035925791307, 1.0194312342199303,
        73.13594102592681, 33.90427473913698, 1.7798454536689583,
        16.79069217506678, 10.143203459467498, 2.78850831094937,
        36.769471333764606, 3.2115400027014904, 13.415415298010092,
        58.709776889457565, 5.631605414335738, 109.63644825434017,
        57.146572273001446, 4.904498023785656, 23.957782910543298,
        29.717470513706303],
       [57.46187961960476, 5.7782268196714455, 23.23027069622881,
        26.36498084485093, 2.378311147592842, 159.22683856129012,
        9.332830688211182, 3.545196781616775, 1.1274135020025375,
        149.76508832902923, 55.48358082573632, 3.206019413263825,
        19.86003066770919, 12.696928271895183, 2.7770008675577067,
        44.97419690754983, 3.189074397724565, 13.60297176889464,
        105.78092006611087, 20.43464752983805, 110.63395833353509,
        53.45420808302541, 4.904498023785656, 33.53974973921886,
        76.97308040665638]], dtype=object)

mean = np.mean(efficiency_matrix)

std = np.std(efficiency_matrix)

print("平均值：", mean)
print("标准差：", std)



平均值： 32.363850951445144
标准差： 43.30872666985573


In [6]:
fitness_matrix = np.array([[8.770676292790775, 0.4154285773900174, 0.6669313489301868,
        3.901153202958363, 0.7989037016497241, 5.258522669301234,
        0.3689786789309141, 0.49746411622571435, 0.6603007374610418,
        1.2599398827542083, 1.2487093223031756, 1.5352007262536747,
        4.238860877662644, 0.7311960637184372, 0.32300784765761614,
        5.267268249581435, 0.380499785209238, 2.817979545958534,
        2.099531828682503, 2.4811259014954157, 10.141889571996268,
        8.74535362544958, 0.33357735222421314, 5.174848340361833,
        0.7562117701403028],
       [6.7719743095361205, 0.39103667688223853, 0.4453263817322554,
        1.0094524060206356, 0.45782031736432643, 0.8502609132128915,
        0.3473546785569741, 0.44360821860449123, 0.659905064626487,
        0.42787486526315943, 0.4276842865470231, 0.6424778796124077,
        2.3742768434424413, 0.5258727347724826, 0.32466956159387295,
        5.252853170558212, 0.3651001603382401, 3.102345880235173,
        0.42787486526315943, 0.5919575969144095, 6.0998956772866215,
        8.491882895555655, 0.3335229828572661, 3.3012651846548424,
        0.4061978008800798],
       [8.422180509658556, 0.4046891708171827, 0.44791815089666903,
        2.227584558258032, 0.566277910600732, 1.1638389995654959,
        0.3520666272848551, 0.43988556065509754, 0.616675424700787,
        0.4658941574393115, 0.817578966587973, 1.1065269352824776,
        2.7634359126846535, 0.7036515825580991, 0.32190124215565424,
        6.442024737726695, 0.38075622800392034, 2.9789608884748184,
        0.4642671131431766, 1.317950136600006, 10.06526081214582,
        8.024739580317204, 0.3335229828572661, 3.9933724959029298,
        0.52227016783747],
       [8.051078355454145, 0.3970461009806676, 0.45638583846142566,
        1.5965142955564824, 0.5640160756355317, 0.8990718596533247,
        0.34875369314494686, 0.43883948395617467, 0.5407941627728008,
        0.4334235787793254, 0.5870752377645668, 0.9789936702925034,
        3.4668223767023467, 0.782011491545018, 0.3224975064809974,
        4.847013238613298, 0.3802968834279677, 3.0304908647178452,
        0.4378680873625151, 0.7319318216853916, 10.117676579369173,
        9.057889851085537, 0.3335229828572661, 3.5385275051629828,
        0.4792365125366374],
       [8.220991108473605, 0.392541578092409, 0.45795703252943043,
        3.908245063091535, 0.6832705392062438, 5.4863507184371825,
        0.3471246010180185, 0.4604189953391703, 0.5980772615631653,
        0.8875488528514087, 0.9607353838393253, 1.763452386245186,
        4.10055749953234, 0.9788962486675904, 0.3211666437451,
        5.928573892949048, 0.3776366022077861, 3.0728591521698667,
        0.7889331488349147, 2.65586235035919, 10.209730677496669,
        8.472639908829656, 0.3335229828572661, 4.95376919524051,
        1.2413004867380801]], dtype=object)

# configuration of fine optimization

In [7]:
# 获取矩阵中所有非 None 的值和它们的坐标
all_values_with_coords = []
for i in range(len(efficiency_matrix)):
    for j in range(len(efficiency_matrix[0])):
        value = efficiency_matrix[i][j]
        if value is not None:
            all_values_with_coords.append(((i, j), value))

# 按值降序排序
sorted_values = sorted(all_values_with_coords, key=lambda x: x[1], reverse=True)

# 计算前 20% 的数量（至少选1个）
top_k = max(1, int(len(sorted_values) * 0.1))
# 取前 20% 个坐标
efficiency_coarse_best = [coord for coord, val in sorted_values[:top_k]]

logging.info(f"fitness_coarse_best {efficiency_coarse_best}")
logging.info(f"fitness_coarse_best values {sorted_values[:top_k]}")


In [8]:
coarse_best = efficiency_coarse_best
coarse_best

[(0, 18),
 (0, 9),
 (4, 5),
 (0, 5),
 (4, 9),
 (4, 20),
 (0, 20),
 (3, 20),
 (2, 20),
 (4, 18),
 (2, 9),
 (4, 24)]

In [9]:
efficiency_matrix_select = efficiency_matrix

# enter fine optimization stage

In [ ]:
final_optimized_results = []  # 用来记录每个 coarse_best 的最优结果

for rewardfunc_index, morphology_index in coarse_best:
    
    morphology = morphology_list[morphology_index]
    parameter = parameter_list[morphology_index]
    rewardfunc = rewardfunc_list[rewardfunc_index]
    
    best_efficiency = efficiency_matrix_select[rewardfunc_index][morphology_index]
    best_fitness = fitness_matrix[rewardfunc_index][morphology_index]
    best_morphology = morphology
    best_parameter = parameter
    best_rewardfunc = rewardfunc
    best_material = compute_ant_volume(parameter)
    
    
    logging.info(f"Initial morphology:{morphology}")
    logging.info(f"Initial parameter:{parameter}" )
    logging.info(f"Initial rewardfunc:{rewardfunc}" )
    logging.info(f"Initial fitness:{best_fitness}" )
    logging.info(f"Initial efficiency:{best_efficiency}" )
    iteration = 0

    while True:
        improved = False  # 标记是否有改进，方便控制循环

        designer = DGA()
        
         # -------- 优化 morphology --------
        improved_morphology, improved_parameter = designer.improve_morphology(
            best_parameter,
            parameter_list,
            efficiency_matrix_select[rewardfunc_index, :],
            folder_name,
            rewardfunc_index, 
            morphology_index,
            iteration
            
        )

        shutil.copy(improved_morphology, "GPTAnt.xml")
        shutil.copy(best_rewardfunc, "GPTrewardfunc.py")
        
        import GPTrewardfunc
        importlib.reload(GPTrewardfunc)  # 重新加载模块
        from GPTrewardfunc import _get_rew
        GPTAntEnv._get_rew = _get_rew
        
        model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=5e5)
        improved_fitness, _ = Eva(model_path)
        improved_material = compute_ant_volume(improved_parameter)
        improved_efficiency = improved_fitness / improved_material

        if improved_efficiency > best_efficiency:

            best_fitness = improved_fitness
            best_morphology = improved_morphology
            best_parameter = improved_parameter
            best_material = improved_material
            best_efficiency = improved_efficiency
            improved = True
            iteration +=1
            logging.info(f"Morphology optimization improved iteration {iteration}: material={improved_material}, fitness={improved_fitness}, efficiency={improved_efficiency}")

        # -------- 没有进一步改进，跳出循环 --------
        if not improved:
            logging.info("Not improved Morphology!")
            logging.info("____________________________________________")
            break
            
            
        # -------- 优化 reward function --------
        improved_rewardfunc = designer.improve_rewardfunc(
            best_rewardfunc,
            rewardfunc_list,
            efficiency_matrix_select[:, morphology_index],
            folder_name,
            rewardfunc_index, 
            morphology_index,
            iteration
        )

        shutil.copy(best_morphology, "GPTAnt.xml")
        shutil.copy(improved_rewardfunc, "GPTrewardfunc.py")
        
        import GPTrewardfunc
        importlib.reload(GPTrewardfunc)  # 重新加载模块
        from GPTrewardfunc import _get_rew
        GPTAntEnv._get_rew = _get_rew
        
        model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=5e5)
        improved_fitness, _ = Eva(model_path)
        improved_material = compute_ant_volume(best_parameter)
        improved_efficiency = improved_fitness / improved_material


        if improved_efficiency > best_efficiency:
            best_fitness = improved_fitness
            best_rewardfunc = improved_rewardfunc
            best_material = improved_material
            best_efficiency = improved_efficiency
            improved = True
            iteration +=1
            logging.info(f"Reward optimization improved iteration {iteration}: material={improved_material}, fitness={improved_fitness}, efficiency={improved_efficiency}")
        
        if not improved:
            logging.info("Not improved Reward!")
            logging.info("____________________________________________")
            break
            

            
    # 保存当前 coarse_best 的最终最优结果
    final_optimized_results.append({
        "best_morphology": best_morphology,
        "best_parameter": best_parameter,
        "best_rewardfunc": best_rewardfunc,
        "best_fitness": best_fitness,
        "best_material": best_material,
        "best_efficiency": best_efficiency,
        "best_iteration":iteration
    })

    logging.info(f"Final optimized result: rewardfunc_index{rewardfunc_index} morphology_index{morphology_index}")
    logging.info(f"  Morphology: {best_morphology}")
    logging.info(f"  Parameter: {best_parameter}")
    logging.info(f"  Rewardfunc: {best_rewardfunc}")
    logging.info(f"  Fitness: {best_fitness}")
    logging.info(f"  Material: {best_material}")
    logging.info(f"  Efficiency: {best_efficiency}")
    logging.info("____________________________________________")

params: [0.1, 0.2, 0.05, 0.15, 0.05, 0.2, 0.05, 0.02, 0.02, 0.02]
ChatCompletion(id='chatcmpl-BSFxzswtvBlf3yFELhoqIFptxXLbq', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "parameters": [0.12, 0.22, 0.06, 0.17, 0.06, 0.22, 0.06, 0.018, 0.018, 0.018],\n  "description": "Optimization by slight increment on the best-performing parameters to enhance jump while keeping material costs low."\n}', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1746074259, model='gpt-4-turbo-2024-04-09', object='chat.completion', service_tier=None, system_fingerprint='fp_5603ee5e2e', usage=CompletionUsage(completion_tokens=81, prompt_tokens=4070, total_tokens=4151, completion_tokens_details=None, prompt_tokens_details=None))
[0.12, 0.22, 0.06, 0.17, 0.06, 0.22, 0.06, 0.018, 0.018, 0.018]
Successfully saved GPTAnt_refine_0_18_0.xml


/root/miniconda3/envs/robodesign/lib/python3.8/site-packages/stable_baselines3/common/save_util.py:284: UserWarning: Path 'results/noDiv_m25_r5/fine' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
params: [0.12, 0.22, 0.06, 0.17, 0.06, 0.22, 0.06, 0.018, 0.018, 0.018]
params: [0.08, 0.1, 0.04, 0.15, 0.05, 0.2, 0.05, 0.025, 0.025, 0.02]
ChatCompletion(id='chatcmpl-BSG4Us5gVeC9yIwQ2ogHseOeZIjRj', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "parameters": [0.08, 0.11, 0.04, 0.16, 0.06, 0.21, 0.06, 0.025, 0.025, 0.02],\n  "description": "Optimized parameters focusing on minimal material usage while ensuring effective limb length and joint functionality for maximum jumping performance."\n}', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1746074662, model='gpt-4-turbo-2024-04-0

In [15]:
logging.info(f"{final_optimized_results}")

# logging.info(f"fine optimization end: best material cost: {best_material}  fitness: {improved_fitness} merterial_efficiency: {improved_material_efficiency}")

In [13]:
final_optimized_results

[{'best_morphology': 'results/noDiv_m25_r5/assets/GPTAnt_18.xml',
  'best_parameter': [0.1, 0.2, 0.05, 0.15, 0.05, 0.2, 0.05, 0.02, 0.02, 0.02],
  'best_rewardfunc': 'results/noDiv_m25_r5/env/GPTrewardfunc_0.py',
  'best_fitness': 2.099531828682503,
  'best_material': 0.0074581800606560045,
  'best_efficiency': 281.5072593591731,
  'best_iteration': 0},
 {'best_morphology': 'results/noDiv_m25_r5/assets/GPTAnt_9.xml',
  'best_parameter': [0.08,
   0.1,
   0.04,
   0.15,
   0.05,
   0.2,
   0.05,
   0.025,
   0.025,
   0.02],
  'best_rewardfunc': 'results/noDiv_m25_r5/env/GPTrewardfunc_0.py',
  'best_fitness': 1.2599398827542083,
  'best_material': 0.005926273357523027,
  'best_efficiency': 212.60239053178248,
  'best_iteration': 0},
 {'best_morphology': 'results/noDiv_m25_r5/assets/GPTAnt_5.xml',
  'best_parameter': [0.1, 0.2, 0.1, 0.3, 0.1, 0.2, 0.1, 0.05, 0.05, 0.05],
  'best_rewardfunc': 'results/noDiv_m25_r5/env/GPTrewardfunc_4.py',
  'best_fitness': 5.4863507184371825,
  'best_mate

In [14]:
best_efficiency

132.38918964657012